# Ray RLlib Training for Zelda Oracle of Seasons

This notebook deploys a Ray cluster on OpenShift/Kubernetes and submits a distributed training job.

Based on the Double Dragon KubeRay implementation.


In [ ]:
!pip install codeflare-sdk


In [ ]:
# Updated imports for newer codeflare_sdk versions
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
from codeflare_sdk.cluster.auth import TokenAuthentication
import os
import time


In [ ]:
# Authenticate with OpenShift
# Get your token: oc whoami -t
# Get your server: oc cluster-info

auth = TokenAuthentication(
    token = 'YOUR_TOKEN_HERE',  # Replace with: oc whoami -t
    server = 'YOUR_SERVER_HERE',  # Replace with: oc cluster-info
    skip_tls=False
)
auth.login()


In [ ]:
# First, let's check what parameters ClusterConfiguration actually accepts
import inspect
sig = inspect.signature(ClusterConfiguration.__init__)
print("Available ClusterConfiguration parameters:")
for param_name, param in sig.parameters.items():
    if param_name != 'self':
        default = param.default if param.default != inspect.Parameter.empty else "REQUIRED"
        print(f"  {param_name}: {default}")


In [ ]:
# Configure Ray Cluster
# ‚úÖ REUSING the Double Dragon image - it already has everything we need!
# üéÆ PyBoy emulator works great on CPU - no GPUs needed!

# Try removing local_queue if Kueue isn't configured
cluster = Cluster(ClusterConfiguration(
    name='zelda-rl',
    namespace='zelda-hybrid-rl-llm',
    num_workers=3,
    
    # CPU resources (applies to both head and workers)
    min_cpus=8,
    max_cpus=8,
    
    # Memory resources in GB (applies to both head and workers)
    min_memory=12,
    max_memory=16,
    
    # No GPUs needed - PyBoy runs great on CPU!
    num_gpus=0,
    
    # ‚úÖ Reusing DD image - has Ray, PyTorch, PyBoy, all dependencies
    image="quay.io/cnuland/dd-kuberay-worker:latest",
    
    # Remove local_queue if Kueue is not set up
    # local_queue="user-queue",  # Comment this out if you don't have Kueue
))

print(f"Cluster configuration created for: {cluster.config.name}")
print(f"Namespace: {cluster.config.namespace}")
print(f"Workers: {cluster.config.num_workers}")
print(f"Image: {cluster.config.image}")


In [ ]:
# Create the Ray cluster
print("Creating Ray cluster...")
cluster.up()

# Check status immediately
print("\nChecking cluster status...")
status_info = cluster.status()
print(f"Status: {status_info}")

# Try to wait for ready (with better error handling)
try:
    print("\nWaiting for cluster to be ready...")
    cluster.wait_ready(timeout=600)  # 10 minute timeout
    print("‚úÖ Cluster is ready!")
except Exception as e:
    print(f"‚ö†Ô∏è Error waiting for cluster: {e}")
    print("\nCluster details:")
    print(cluster.details())


In [ ]:
# Diagnostic: Check if RayCluster resource was created
import subprocess

print("üîç Checking if RayCluster resource was created...")
try:
    result = subprocess.run(
        ["oc", "get", "raycluster", "-n", "zelda-hybrid-rl-llm"],
        capture_output=True, text=True, timeout=10
    )
    print(result.stdout if result.returncode == 0 else result.stderr)
    
    print("\nüîç Checking pods in namespace...")
    result2 = subprocess.run(
        ["oc", "get", "pods", "-n", "zelda-hybrid-rl-llm"],
        capture_output=True, text=True, timeout=10
    )
    print(result2.stdout if result2.returncode == 0 else result2.stderr)
    
except Exception as e:
    print(f"Error running diagnostic commands: {e}")
    print("You may need to manually check: oc get raycluster -n zelda-hybrid-rl-llm")


In [ ]:
# Get cluster details and submit job
clusterDetails = cluster.details()
print(f"Ray Dashboard URL: {clusterDetails.dashboard}")

client = cluster.job_client

# Configure environment variables
env_vars = {
    # S3 Storage (MinIO)
    'S3_ACCESS_KEY_ID': 'YOUR_S3_KEY',
    'S3_SECRET_ACCESS_KEY': 'YOUR_S3_SECRET',
    'S3_REGION_NAME': 'region',
    'S3_ENDPOINT_URL': 'YOUR_S3_ENDPOINT',
    'S3_BUCKET_NAME': 'YOUR_BUCKET_NAME',
    
    # LLM endpoint (assumes LLM service deployed in cluster)
    'LLM_ENDPOINT': 'http://llama4-scout-service:8000/v1/chat/completions',
}

# Submit training job
# Ray's working_dir uploads our Zelda code to all workers!
submission_id = client.submit_job(
    entrypoint="python run-ray-zelda.py",
    runtime_env={
        "env_vars": env_vars,
        'working_dir': './',  # Uploads our Zelda code, configs, and ROMs
        'pip': [],  # DD image already has everything!
        "excludes": ["*.sh", "*.ipynb", "*.md", "__pycache__", "*.pyc", "checkpoints/", "training_runs/"],
    }
)

print(f"\n‚úÖ Job submitted successfully!")
print(f"Submission ID: {submission_id}")
print(f"Monitor at: {clusterDetails.dashboard}")


In [ ]:
# Monitor job status
from ray.job_submission import JobStatus

while True:
    status = client.get_job_status(submission_id)
    print(f"Job status: {status}")
    
    if status in [JobStatus.SUCCEEDED, JobStatus.FAILED, JobStatus.STOPPED]:
        print(f"\nüìä Final job logs:\n{client.get_job_logs(submission_id)}")
        break
    
    time.sleep(30)  # Check every 30 seconds


In [ ]:
# Get job logs (anytime)
logs = client.get_job_logs(submission_id)
print(logs)


In [ ]:
# Clean up (when training is complete)
# cluster.down()
